## Import the Libraries

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import math
from scipy import stats

## Load the Tickers

In [2]:
tickers = pd.read_csv('top_50_indian_stocks.csv')
tickers.head()

,Ticker,Company Name
0,RELIANCE.NS,Reliance Industries
1,TCS.NS,Tata Consultancy Services
2,HDFCBANK.NS,HDFC Bank
3,INFY.NS,Infosys
4,ICICIBANK.NS,ICICI Bank


## Weighted Scoring Model for finding the final score

In [3]:
def create_dividend_df(tickers):
    columns = [
        "Ticker", 
        "Dividend Yield(%)", 
        "Dividend Rate",
        "Payout Ratio(%)", 
        "5 Year Avg Dividend Yield(%)", 
        "Earning Growth(%)"
        
    ]
    
    dividend_df = pd.DataFrame(columns=columns)
    
    for stock in tickers:
        ticker = yf.Ticker(stock)
        info = ticker.info 
        
        dividend_yield = info.get("dividendYield", np.nan) * 100 if info.get("dividendYield") else np.nan
        dividend_rate = info.get("dividendRate", np.nan) 
        payout_ratio = info.get("payoutRatio", np.nan) * 100 if info.get("payoutRatio") else np.nan
        five_year_avg_dividend_yield = info.get("fiveYearAvgDividendYield", np.nan) * 100 if info.get("fiveYearAvgDividendYield") else np.nan
        earning_growth = info.get("earningsGrowth", np.nan) * 100 if info.get("earningsGrowth") else np.nan
        
        dividend_df.loc[len(dividend_df)] = [stock, dividend_yield, dividend_rate, payout_ratio, five_year_avg_dividend_yield, earning_growth]
        
    ## we have got our df with dividend information
    
    numeric_cols = [
          "Dividend Yield(%)", 
        "Dividend Rate",
        "Payout Ratio(%)", 
        "5 Year Avg Dividend Yield(%)", 
        "Earning Growth(%)"
    ]
    

    for col in numeric_cols:
        if col == "Payout Ratio":
            dividend_df[col+ " Normalised"] = 1 - ((dividend_df[col] - dividend_df[col].min()) / (dividend_df[col].max() - dividend_df[col].min()))
        else:
            dividend_df[col+ " Normalised"] = (dividend_df[col] - dividend_df[col].min()) / (dividend_df[col].max() - dividend_df[col].min())
        
    return dividend_df

In [4]:
tickers_list = tickers["Ticker"].values.tolist()
dividend_df = create_dividend_df(tickers_list)
dividend_df

,Ticker,Dividend Yield(%),Dividend Rate,Payout Ratio(%),5 Year Avg Dividend Yield(%),Earning Growth(%),Dividend Yield(%) Normalised,Dividend Rate Normalised,Payout Ratio(%) Normalised,5 Year Avg Dividend Yield(%) Normalised,Earning Growth(%) Normalised
0,RELIANCE.NS,0.390000,5.00,9.960000,44.0,-4.8,0.032850,0.036145,0.058941,0.042411,0.109364
1,TCS.NS,0.890000,40.00,42.810002,128.0,6.2,0.081159,0.317269,0.277722,0.136161,0.124535
2,HDFCBANK.NS,1.040000,19.50,21.520000,67.0,39.0,0.095652,0.152610,0.135931,0.068080,0.169770
3,INFY.NS,2.050000,41.00,59.240000,213.0,4.2,0.193237,0.325301,0.387146,0.231027,0.121776
4,ICICIBANK.NS,0.740000,10.00,15.120000,99.0,18.2,0.066667,0.076305,0.093307,0.103795,0.141084
5,HINDUNILVR.NS,1.800000,43.00,96.089995,144.0,-2.4,0.169082,0.341365,0.632567,0.154018,0.112674
6,SBIN.NS,1.590000,13.70,17.090000,139.0,22.9,0.148792,0.106024,0.106427,0.148438,0.147566
7,BAJFINANCE.NS,0.500000,36.00,14.479999,24.0,10.4,0.043478,0.285141,0.089044,0.020089,0.130327
8,BHARTIARTL.NS,0.480000,8.00,38.680000,80.0,159.7,0.041546,0.060241,0.250216,0.082589,0.336229
9,ITC.NS,3.190000,15.00,83.739996,404.0,1.8,0.303382,0.116466,0.550316,0.444196,0.118466


In [5]:
weights = {
        "Dividend Yield(%) Normalised": 0.3,
        "Dividend Rate Normalised": 0.2,
        "Payout Ratio(%) Normalised": 0.2,
        "5 Year Avg Dividend Yield(%) Normalised": 0.2,
        "Earning Growth(%) Normalised": 0.1
    }

dividend_df["Dividend Score"] = dividend_df[[col for col in weights.keys()]].mul(list(weights.values())).sum(axis = 1)
dividend_df

,Ticker,Dividend Yield(%),Dividend Rate,Payout Ratio(%),5 Year Avg Dividend Yield(%),Earning Growth(%),Dividend Yield(%) Normalised,Dividend Rate Normalised,Payout Ratio(%) Normalised,5 Year Avg Dividend Yield(%) Normalised,Earning Growth(%) Normalised,Dividend Score
0,RELIANCE.NS,0.390000,5.00,9.960000,44.0,-4.8,0.032850,0.036145,0.058941,0.042411,0.109364,0.048291
1,TCS.NS,0.890000,40.00,42.810002,128.0,6.2,0.081159,0.317269,0.277722,0.136161,0.124535,0.183032
2,HDFCBANK.NS,1.040000,19.50,21.520000,67.0,39.0,0.095652,0.152610,0.135931,0.068080,0.169770,0.116997
3,INFY.NS,2.050000,41.00,59.240000,213.0,4.2,0.193237,0.325301,0.387146,0.231027,0.121776,0.258843
4,ICICIBANK.NS,0.740000,10.00,15.120000,99.0,18.2,0.066667,0.076305,0.093307,0.103795,0.141084,0.088790
5,HINDUNILVR.NS,1.800000,43.00,96.089995,144.0,-2.4,0.169082,0.341365,0.632567,0.154018,0.112674,0.287582
6,SBIN.NS,1.590000,13.70,17.090000,139.0,22.9,0.148792,0.106024,0.106427,0.148438,0.147566,0.131572
7,BAJFINANCE.NS,0.500000,36.00,14.479999,24.0,10.4,0.043478,0.285141,0.089044,0.020089,0.130327,0.104931
8,BHARTIARTL.NS,0.480000,8.00,38.680000,80.0,159.7,0.041546,0.060241,0.250216,0.082589,0.336229,0.124696
9,ITC.NS,3.190000,15.00,83.739996,404.0,1.8,0.303382,0.116466,0.550316,0.444196,0.118466,0.325057


## Sort according to Dividend Score

In [6]:
dividend_df = dividend_df.sort_values(by = "Dividend Score", ascending = False)
dividend_df.head(10)

,Ticker,Dividend Yield(%),Dividend Rate,Payout Ratio(%),5 Year Avg Dividend Yield(%),Earning Growth(%),Dividend Yield(%) Normalised,Dividend Rate Normalised,Payout Ratio(%) Normalised,5 Year Avg Dividend Yield(%) Normalised,Earning Growth(%) Normalised,Dividend Score
30,COALINDIA.NS,10.050000,41.25,43.560000,902.0,-7.4,0.966184,0.327309,0.282717,1.000000,0.105779,0.622438
45,IOC.NS,10.399999,15.00,94.269997,718.0,NaN,1.000000,0.116466,0.620446,0.794643,NaN,0.606311
31,BPCL.NS,7.620000,23.00,68.360000,563.0,-72.2,0.731401,0.180723,0.447885,0.621652,0.016412,0.471113
42,HEROMOTOCO.NS,2.500000,115.00,53.990000,323.0,47.1,0.236715,0.919679,0.352181,0.353795,0.180941,0.414239
20,ONGC.NS,7.180000,18.25,37.500000,504.0,-25.2,0.688889,0.142570,0.242358,0.555804,0.081230,0.402936
22,POWERGRID.NS,4.720000,15.75,67.090000,514.0,0.5,0.451208,0.122490,0.439427,0.566964,0.116674,0.372806
28,TECHM.NS,2.390000,43.00,108.430000,221.0,153.1,0.226087,0.341365,0.714752,0.239955,0.327127,0.359753
41,TATASTEEL.NS,2.420000,3.60,151.260000,293.0,NaN,0.228986,0.024900,1.000000,0.320312,NaN,0.337738
13,HCLTECH.NS,2.440000,48.00,87.100000,281.0,9.8,0.230918,0.381526,0.572694,0.306920,0.129499,0.334453
9,ITC.NS,3.190000,15.00,83.739996,404.0,1.8,0.303382,0.116466,0.550316,0.444196,0.118466,0.325057
